# Buckets creation, and notifications configuration

First, of all, if you've never used a Jupyter notebook before, here are a few information:
- A notebook is an environment where you have *cells* that can display formatted text (like the present cell), or code (as you will see below).
- Code cells contain Python code that can be run interactively. Thats means you can modify the code, as you will do in the first code cell, then run it. The code will not run on you computer or in the browser, but directly on the server you are connected to.
- To run a code cell, just select it (click in the cell, or on the left side of it), and click the "Run/Play" button from the toolbar.
- You can also press CTRL+Enter to run a cell, or Shift+Enter to run the cell and automatically select the following one.
- You can navigate between cells with Up and Down arrows (on your keyboard or in the toolbar).

With this (very) quick turorial, you should be able to run the different parts of this notebook. If you still have a doubt, here is a longer tutorial: https://www.codecademy.com/articles/how-to-use-jupyter-notebooks

# Preparatory steps

## ---==== REALLY IMPORTANT STEP! ====---
### Please read carefully and make all the requested changes. There are 4 to make.
## Parameters

We will first start by setting some paramaters that will be used in this notebook. Replace the one that are indicated with the needed values, and run the cell.

In [3]:
import os
# Warning!!!: On all those variables, don't remove the starting/ending quotes when doing copy/paste

# Enter you bucket base name. That's the same one you have put in the config map. It should be identical as your namespace if you have followed instructions.
bucket_base_name = 'images'

# Enter you Access and Secret keys. They're the ones that were displayed in the instructions.
aws_access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')

print(aws_access_key_id)
print(aws_secret_access_key)

# Do not change this value, this is the internal location for the RGW
endpoint_url = os.getenv('S3_ENDPOINT_URL')

print(endpoint_url)

TA9TYEX3SGFIN0RFSPFQ
wSnnSvAb0sKwKX4jtxWEpFcwjRcMVIUdHpAZApEt
http://ceph-nano-0


## Imports
Of course we'll need some libraries, so import them by running the cell.

In [32]:
import boto3
import json
import botocore
import argparse

## S3 and SNS connections
The boto3 is the standard library from AWS to interact with all their services. As Ceph is compatible with S3 and SNS, we can directly use this library to interact with the storage. So first, let's create the clients (you can see we are using some parameters we defined earlier).

In [36]:
s3 = boto3.client('s3', '',
                endpoint_url = os.getenv('S3_ENDPOINT_URL'),
                aws_access_key_id = aws_access_key_id,
                aws_secret_access_key = aws_secret_access_key,
                config=botocore.client.Config(signature_version = 's3'))


## Create buckets
Now that we can connect to the storage, we can create our buckets. Run the first cell, which will define a "creation function" (an S3 API call using the client we created). Then the second cell that will create the 3 buckets we will need.

In [37]:
def create_bucket(bucket_name):
    result = s3.create_bucket(Bucket=bucket_name)
    return result

In [38]:
create_bucket(bucket_base_name)
create_bucket(bucket_base_name+'-processed')
create_bucket(bucket_base_name+'-anonymized')

{'ResponseMetadata': {'RequestId': 'tx000000000000000000009-006088ca15-1010-default',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-request-id': 'tx000000000000000000009-006088ca15-1010-default',
   'content-length': '0',
   'date': 'Wed, 28 Apr 2021 02:36:05 GMT',
   'connection': 'Keep-Alive'},
  'RetryAttempts': 0}}

### Verification
As the previous output may have been cryptic (and anyway it's always good to check), let's list all the buckets and verify the indeed have been created.

In [39]:
for bucket in s3.list_buckets()['Buckets']:
    print(bucket['Name'])

bucket-base-user1
bucket-base-user1-anonymized
bucket-base-user1-processed
covid


## Make buckets public read
Our Grafana dashboard will display the last image from each bucket. Instead of setting up a dedicated web server, we can directly query our object stores to retrieve the images. For this to work we have to make our bucket "public-readable". This is done by applying to each this bucket policy.

In [40]:
for bucket in s3.list_buckets()['Buckets']:
    bucket_policy = {
                      "Version":"2012-10-17",
                      "Statement":[
                        {
                          "Sid":"AddPerm",
                          "Effect":"Allow",
                          "Principal": "*",
                          "Action":["s3:GetObject"],
                          "Resource":["arn:aws:s3:::{0}/*".format(bucket['Name'])]
                        }
                      ]
                    }
    bucket_policy = json.dumps(bucket_policy)
    s3.put_bucket_policy(Bucket=bucket['Name'], Policy=bucket_policy)